In [1]:
import os
import torch
import json
import random
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import torch.nn as nn
import torch.nn.functional as F

Set seed for reproducibility

In [2]:
def set_seed(seed=7):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(7)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [4]:
class TransEModel(nn.Module):
    def __init__(self, num_entities, num_relations, embedding_dim):
        super(TransEModel, self).__init__()
        self.entity_embeddings = nn.Embedding(num_entities, embedding_dim)
        self.relation_embeddings = nn.Embedding(num_relations, embedding_dim)
        self.embedding_dim = embedding_dim
        self.init_weights()

    def init_weights(self):
        nn.init.xavier_uniform_(self.entity_embeddings.weight.data)
        nn.init.xavier_uniform_(self.relation_embeddings.weight.data)

    def forward(self, head, relation, tail):
        h = self.entity_embeddings(head)
        r = self.relation_embeddings(relation)
        t = self.entity_embeddings(tail)
        return h + r - t

    def score(self, head, relation, tail):
        return torch.norm(self.forward(head, relation, tail), p=1, dim=1)

In [5]:
from torch.utils.data import DataLoader, TensorDataset

Load datasets

In [6]:
train_data = torch.load("models/transe_train.pt")
val_data = torch.load("models/transe_val.pt")

C:\Users\mca\AppData\Local\Temp\ipykernel_13344\205628700.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_data = torch.load("models/transe_train.pt")
C:\Users\mca\

Load vocab

In [7]:
entity_vocab = torch.load("models/entity_vocab.pt")
relation_vocab = torch.load("models/relation_vocab.pt")

C:\Users\mca\AppData\Local\Temp\ipykernel_13344\3983285956.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  entity_vocab = torch.load("models/entity_vocab.pt")
C:\Users\m

In [8]:
print(type(train_data))
print(len(train_data))
print(type(train_data[0]))

<class 'list'>
10132
<class 'tuple'>


In [9]:
from torch.utils.data import TensorDataset, DataLoader

In [10]:

# Unpack triplets into three lists
heads, relations, tails = zip(*train_data)  # works because it's a list of tuples

# Convert to tensors
heads = torch.tensor(heads, dtype=torch.long)
relations = torch.tensor(relations, dtype=torch.long)
tails = torch.tensor(tails, dtype=torch.long)

# Now create TensorDataset and DataLoader
train_dataset = TensorDataset(heads, relations, tails)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)


In [11]:
val_heads, val_relations, val_tails = zip(*val_data)
val_dataset = TensorDataset(
    torch.tensor(val_heads, dtype=torch.long),
    torch.tensor(val_relations, dtype=torch.long),
    torch.tensor(val_tails, dtype=torch.long)
)
val_loader = DataLoader(val_dataset, batch_size=512)

In [12]:
num_entities = len(entity_vocab)
num_relations = len(relation_vocab)
embedding_dim = 100 

In [13]:
# Hyperparameters
embedding_dim = 100
num_epochs = 50
learning_rate = 0.001
margin = 1.0

In [14]:
# Model & Optimizer
model = TransEModel(num_entities, num_relations, embedding_dim).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [15]:
# Margin Ranking Loss
criterion = nn.MarginRankingLoss(margin=margin)

In [16]:
# Evaluation function
def evaluate(model, val_loader):
    model.eval()
    with torch.no_grad():
        total_loss = 0
        for h, r, t in val_loader:
            h, r, t = h.to(device), r.to(device), t.to(device)
            # Corrupt tail by shuffling
            t_corrupt = t[torch.randperm(t.size(0))].to(device)
            score_pos = model.score(h, r, t)
            score_neg = model.score(h, r, t_corrupt)
            target = torch.ones_like(score_pos)
            loss = criterion(score_pos, score_neg, target)
            total_loss += loss.item()
        return total_loss / len(val_loader)

In [17]:
# Training loop
best_val_loss = float("inf")

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for h, r, t in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        h, r, t = h.to(device), r.to(device), t.to(device)

        # Corrupt tail
        t_corrupt = t[torch.randperm(t.size(0))].to(device)

        # Scores
        score_pos = model.score(h, r, t)
        score_neg = model.score(h, r, t_corrupt)

        # Loss
        target = torch.ones_like(score_pos)
        loss = criterion(score_pos, score_neg, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)
    val_loss = evaluate(model, val_loader)

    print(f"Epoch {epoch+1}: Train Loss = {avg_train_loss:.4f}, Val Loss = {val_loss:.4f}")

    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), f"models/transe_model_valLoss_{val_loss:.4f}.pt")
        print("✅ Saved best model.")


Epoch 1/50: 100%|██████████| 20/20 [00:00<00:00, 44.01it/s]


Epoch 1: Train Loss = 0.8575, Val Loss = 0.7216
✅ Saved best model.


Epoch 2/50: 100%|██████████| 20/20 [00:00<00:00, 221.33it/s]


Epoch 2: Train Loss = 0.5395, Val Loss = 0.5873
✅ Saved best model.


Epoch 3/50: 100%|██████████| 20/20 [00:00<00:00, 210.52it/s]


...

Epoch 48: Train Loss = 0.0371, Val Loss = 0.3945


Epoch 49/50: 100%|██████████| 20/20 [00:00<00:00, 112.75it/s]


Epoch 49: Train Loss = 0.0343, Val Loss = 0.4031


Epoch 50/50: 100%|██████████| 20/20 [00:00<00:00, 186.14it/s]

Epoch 50: Train Loss = 0.0408, Val Loss = 0.4036
